In [1]:
# Import Packages
from scipy.optimize import fsolve
import numpy as np
import pint; u = pint.UnitRegistry()

In [2]:
# ϕ-ϕ Method

# Written by: Frank T. Mtetwa

# Eqautions were taken from Chapter 12 of Fundamentals of Chemical Engineering Thermodynamics Kevin D. Dahm and Donald P. Visco Jr.
# and Chemical, Biochemical and Engineering Thermodynamics Stanlet I. Sandler 4th Edition

## Operating conditions

In [16]:
R = 8.314 # J/mol.K Universal gas constant
T = 20 + 273 # K Temperature
k12 = 0
P1 = 1.5*10**5 # Pa Parital Pressure of Carbon dioxide

## Component Properties

Component properties were taken from DIPPR

In [17]:
# Carbon dioxide
Tc1 = 304.21 # K Critical Temperature
Pc1 = 73.829448*10**5 # Bar Critical Pressure
ω1 = 0.223621 # Acentric Factor

# Carbon disulphide
Tc2 = 552.49 # K Critical Temperature
Pc2 = 73.2893858*10**5 # Bar Critical Pressure
ω2 = 0.084158 # Acentric Factor

## Peng-Robinson EOS

Parameters for individual components

In [18]:
# Carbon dioxide
κ1 = 0.37464+1.54226*ω1-0.26992*ω1**2                    ## 6.7-4
α1 = (1+κ1*(1-np.sqrt(T/Tc1)))**2                        ## 6.7-3
a1 = 0.45724*((R**2*Tc1**2)/Pc1)*α1                      ## 6.7-1
b1 = 0.07780*((R*Tc1)/Pc1)                               ## 6.7-2

# Carbon disulphide
κ2 = 0.37464+1.54226*ω2-0.26992*ω2**2                    ## 6.7-4
α2 = (1+κ2*(1-np.sqrt(T/Tc2)))**2                        ## 6.7-3
a2 = 0.45724*((R**2*Tc2**2)/Pc2)*α2                      ## 6.7-1
b2 = 0.07780*((R*Tc2)/Pc2)                         ## 6.7-2

## Solve 

In [19]:
def phi_phi(X):
    
    x1  = X[0]
    y1  = X[1]
    V_L = X[2]
    V_V = X[3]
    P   = X[4]
   
    # Mole fractions
    x2  = 1 - x1 
    y2  = 1 - y1
    
    # Peng Robinson
    a21 = np.sqrt(a1*a2)*(1-k12)                                    # 11.125
    a12 = a21                                                     
#    b12 = (b1+b2)/2                                                 # 11.126
#    b21 = b12

    am_L = x1**2*a1 + x2**2*a2 + 2*x1*x2*a12                        # 12.54
    bm_L = x1*b1 + x2*b2                                            # 12.55

    am_V = y1**2*a1 + y2**2*a2 + 2*y1*y2*a12                       # 12.56
    bm_V = y1*b1 + y2*b2                                            # 12.57
    
    P1_L = (R*T)/(V_L-bm_L)
    P2_L = am_L/(V_L*(V_L + bm_L) + bm_L*(V_L - bm_L))
    
    P1_V = (R*T)/(V_V-bm_V)
    P2_V = am_L/(V_V*(V_V + bm_V) + bm_L*(V_V - bm_V))
    
    #P = P1/y1
    
    # Compressibility Factors
    Zm_V = (P*V_V)/(R*T)
    Zm_L = (P*V_L)/(R*T)

    # Fugacities
    Part1 = (b1/bm_V)*(Zm_V - 1)
    Part2 = np.log(Zm_V -(bm_V*P)/(R*T))
    Part3 = (am_V/(2*np.sqrt(2)*bm_V*(R*T)))*((2*(y1*a1 + y2*a12)/am_V) - (b1/bm_V))
    Part4 = np.log((Zm_V + (1 + np.sqrt(2))*(bm_V*P/(R*T)))/(Zm_V + (1 - np.sqrt(2))*(bm_V*P/(R*T))))
    
    Part5 = ((b1/bm_L)*(Zm_L - 1))
    Part6 = (np.log(Zm_L -(bm_L*P)/(R*T)))
    Part7 = (am_L/(2*np.sqrt(2)*bm_L*(R*T)))*(2*(x1*a1 + x2*a12)/am_L - b1/bm_L)
    Part8 = np.log((Zm_L + (1 + np.sqrt(2))*(bm_L*P/(R*T)))/(Zm_L + (1 - np.sqrt(2))*(bm_L*P/(R*T))))
    
    Part9 = ((b2/bm_V)*(Zm_V - 1))
    Part10 = (np.log(Zm_V -(bm_V*P)/(R*T)))
    Part11 = (am_V/(2*np.sqrt(2)*bm_V*(R*T)))*(2*(y2*a2 + y1*a12)/am_V - b2/bm_V)
    Part12 = np.log((Zm_V + (1 + np.sqrt(2))*(bm_V*P/(R*T)))/(Zm_V + (1 - np.sqrt(2))*(bm_V*P/(R*T))))
    
    Part13 = ((b2/bm_L)*(Zm_L - 1))
    Part14 = (np.log(Zm_L -(bm_L*P)/(R*T)))
    Part15 = (am_L/(2*np.sqrt(2)*bm_L*(R*T)))*(2*(x2*a2 + x1*a12)/am_L - b2/bm_L)
    Part16 = np.log((Zm_L + (1 + np.sqrt(2))*(bm_L*P/(R*T)))/(Zm_L + (1 - np.sqrt(2))*(bm_L*P/(R*T))))
    
    lnϕ_V1 = Part1 - Part2 - (Part3 * Part4)
    lnϕ_L1 = Part5 - Part6 - (Part7 * Part8)
    lnϕ_V2 = Part9 - Part10 - (Part11 * Part12)
    lnϕ_L2 = Part13 - Part14 - (Part15 * Part16)
    
    ϕ_V1 = np.exp(lnϕ_V1)
    ϕ_L1 = np.exp(lnϕ_L1)   
    ϕ_V2 = np.exp(lnϕ_V2)
    ϕ_L2 = np.exp(lnϕ_L2)
    
    # Equations to be solved
    eqn1 = P - P1_L + P2_L
    eqn2 = P - P1_V + P2_V
    eqn3 = x1*ϕ_L1 - y1*ϕ_V1
    eqn4 = x2*ϕ_L2 - y2*ϕ_V2
    eqn5 = P1 - y1*P
    
    return [eqn1, eqn2, eqn3, eqn4, eqn5]
    

In [20]:
root = fsolve(phi_phi, (0.01, 0.8, 0.00005, 0.012, 200000))#, full_output=True)

In [21]:
print('Liquid composition of 1, x1 = {0:1.5f}'.format(root[0]))
print('Vapour composition of 1, y1 = {0:1.5f}'.format(root[1]))
print('Volume of liquid, V_L = {0:1.7f}'.format(root[2]), 'm^3/mol')
print('Volume of vapour, V_V = {0:1.7f}'.format(root[3]), 'm^3/mol')
print('Total system pressure, P = {0:1.5f}'.format(root[4]), 'Pa')

Liquid composition of 1, x1 = 0.02891
Vapour composition of 1, y1 = 0.78036
Volume of liquid, V_L = 0.0000579 m^3/mol
Volume of vapour, V_V = 0.0119953 m^3/mol
Total system pressure, P = 192219.33243 Pa
